In [72]:
import pandas as pd
import numpy as np

import pickle

In [73]:
books_df = pd.read_csv("Dataset/books_cleaned.csv")
ratings_df = pd.read_csv("Dataset/ratings.csv")

In [74]:
books_df.head()

,Unnamed: 0,book_id,title,authors,year,pages,description,genres,average_rating,ratings_count,books_count,small_image_url
0,0,1,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,2008,374,winning means fame and fortunelosing means cer...,"youngadult, fiction, fantasy, sciencefiction, ...",4.34,4780653,272,https://images.gr-assets.com/books/1447303603s...
1,1,2,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",1997,309,harry potters life is miserable his parents ar...,"fantasy, fiction, youngadult, classics",4.44,4602479,491,https://images.gr-assets.com/books/1474154022s...
2,2,3,"Twilight (Twilight, #1)",Stephenie Meyer,2005,501,about three things i was absolutely positive ...,"youngadult, fantasy, romance, fiction, paranormal",3.57,3866839,226,https://images.gr-assets.com/books/1361039443s...
3,3,4,To Kill a Mockingbird,Harper Lee,1960,324,the unforgettable novel of a childhood in a sl...,"classics, fiction, historicalfiction, youngadult",4.25,3198671,487,https://images.gr-assets.com/books/1361975680s...
4,4,5,The Great Gatsby,F. Scott Fitzgerald,1925,200,alternate cover edition isbn isbn13 the great...,"classics, fiction, historicalfiction, romance",3.89,2683664,1356,https://images.gr-assets.com/books/1490528560s...


In [75]:
ratings_df.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [76]:
books_df.head()

,Unnamed: 0,book_id,title,authors,year,pages,description,genres,average_rating,ratings_count,books_count,small_image_url
0,0,1,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,2008,374,winning means fame and fortunelosing means cer...,"youngadult, fiction, fantasy, sciencefiction, ...",4.34,4780653,272,https://images.gr-assets.com/books/1447303603s...
1,1,2,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",1997,309,harry potters life is miserable his parents ar...,"fantasy, fiction, youngadult, classics",4.44,4602479,491,https://images.gr-assets.com/books/1474154022s...
2,2,3,"Twilight (Twilight, #1)",Stephenie Meyer,2005,501,about three things i was absolutely positive ...,"youngadult, fantasy, romance, fiction, paranormal",3.57,3866839,226,https://images.gr-assets.com/books/1361039443s...
3,3,4,To Kill a Mockingbird,Harper Lee,1960,324,the unforgettable novel of a childhood in a sl...,"classics, fiction, historicalfiction, youngadult",4.25,3198671,487,https://images.gr-assets.com/books/1361975680s...
4,4,5,The Great Gatsby,F. Scott Fitzgerald,1925,200,alternate cover edition isbn isbn13 the great...,"classics, fiction, historicalfiction, romance",3.89,2683664,1356,https://images.gr-assets.com/books/1490528560s...


In [87]:
def get_book_id(book_title):
    book = books_df[books_df['title'] == book_title]
    if len(book) > 0:
        return book['book_id'].iloc[0]
    else:
        return None

In [88]:
get_book_id("The Great Gatsby")

5

In [90]:
def get_new_user_id(title_ratings):
    # Get book IDs for the given book titles
    book_ids = [get_book_id(title) for title in title_ratings.keys()]
    
    # Create a new row for the new user
    new_user_id = ratings_df['user_id'].max() + 1
    new_user_ratings = pd.DataFrame({
        'user_id': [new_user_id] * len(book_ids),
        'book_id': book_ids,
        'rating': list(title_ratings.values())
    })
    
    # Append the new user's ratings to the existing ratings dataframe
    updated_ratings_df = pd.concat([rating_df, new_user_ratings], ignore_index=True)
    
    # Return the ID of the new user and the updated ratings dataframe
    return new_user_id, updated_ratings_df


In [104]:
title_ratings = {"The Hobbit":5, 'To Kill a Mockingbird': 4, 'Pride and Prejudice': 3,"The 3 Mistakes of My Life":5}

# Get the ID of the new user and the updated ratings dataframe
new_user_id, updated_ratings_df = get_new_user_id(title_ratings)

In [105]:
new_user_id

53425

In [94]:
with open("model.pkl", 'rb') as f:
    svd_model = pickle.load(f)

In [109]:
def get_recommendation_svd(user_id, n=10):
    '''Give n recommendation to user_id'''
    
    all_books, user_books =  book_read(user_id)
    next_books = [book for book in all_books if book not in user_books]
    
    if n <= len(next_books):
        ratings = []
        for book in next_books:
            est = svd_model.predict(user_id, book).est
            ratings.append((book, est))
        ratings = sorted(ratings, key=lambda x: x[1], reverse=False)
        book_ids = [id for id, rate in ratings[:n]]
        return books[books.book_id.isin(book_ids)][['book_id', 'title', 'authors', 'year', 'pages', 'description', 'genres', 'average_rating', 'small_image_url']]
    else:
        print('Please reduce your recommendation request')
        return

In [100]:
def book_read(user_id):
    '''Take user_id and return list of book that user has read'''
    books_list = list(books_df['book_id'])
    book_read_list = list(ratings_df['book_id'][ratings_df['user_id'] == user_id])
    return books_list, book_read_list

In [110]:
get_recommendation_svd(new_user_id)

,book_id,title,authors,year,pages,description,genres,average_rating,small_image_url
1323,1409,The 3 Mistakes of My Life,Chetan Bhagat,2008,258,in late2000 a young boy in ahmedabad called go...,"fiction, romance, contemporary, youngadult",2.97,https://s.gr-assets.com/assets/nophoto/book/50...
1683,1793,One Night at the Call Center,Chetan Bhagat,2005,320,press 1 for technical support press 2 for brok...,"fiction, romance, contemporary, youngadult, fa...",2.47,https://images.gr-assets.com/books/1320500924s...
1712,1822,"Revolution 2020: Love, Corruption, Ambition",Chetan Bhagat,2011,296,book summary of revolution 2020 once upon a ti...,"fiction, romance, contemporary, youngadult",3.07,https://images.gr-assets.com/books/1317964666s...
2924,3146,Adultery,Paulo Coelho,2014,272,i want to change i need to change im gradually...,"fiction, romance, contemporary, philosophy",3.01,https://images.gr-assets.com/books/1406247571s...
3707,4045,Half Girlfriend,Chetan Bhagat,2014,260,half girlfriend hindi once upon a time there w...,"fiction, romance, contemporary, youngadult",3.16,https://images.gr-assets.com/books/1407223058s...
4490,4991,Can Love Happen Twice?,Ravinder Singh,2011,224,when ravin first said ‘i love you ’ he meant...,"romance, fiction, contemporary, youngadult",3.31,https://images.gr-assets.com/books/1321016822s...
6550,7636,Of Course I Love You...! Till I Find Someone B...,"Durjoy Datta, Maanvi Ahuja",2008,232,of course i love you is set in delhi 2006–2008...,"romance, fiction, contemporary, youngadult, ch...",3.21,https://images.gr-assets.com/books/1274597947s...
7156,8466,ليتها تقرأ,خالد الباتلي,2010,175,ليتها تقرأ هي ليست كلمات جمعت بين دفتي كتاب، إ...,"poetry, romance, nonfiction",3.04,https://s.gr-assets.com/assets/nophoto/book/50...
7518,9021,Lost,"Gregory Maguire, Douglas Smith",2001,340,at the flat in weatherall walk there was no mi...,"fantasy, fiction, mystery, horror",2.80,https://images.gr-assets.com/books/1407711681s...
8542,4283,Revenge Wears Prada: The Devil Returns (The De...,Lauren Weisberger,2013,400,almost a decade has passed since andy sachs qu...,"chicklit, fiction, romance, contemporary, youn...",2.84,https://s.gr-assets.com/assets/nophoto/book/50...


In [103]:
books_df['title'][:20]

0               The Hunger Games (The Hunger Games, #1)
1     Harry Potter and the Sorcerer's Stone (Harry P...
2                               Twilight (Twilight, #1)
3                                 To Kill a Mockingbird
4                                      The Great Gatsby
5                                The Fault in Our Stars
6                                            The Hobbit
7                                The Catcher in the Rye
8                 Angels & Demons  (Robert Langdon, #1)
9                                   Pride and Prejudice
10                                      The Kite Runner
11                            Divergent (Divergent, #1)
12                            The Diary of a Young Girl
13     The Girl with the Dragon Tattoo (Millennium, #1)
14                 Catching Fire (The Hunger Games, #2)
15    Harry Potter and the Prisoner of Azkaban (Harr...
16                    Mockingjay (The Hunger Games, #3)
17    Harry Potter and the Order of the Phoenix 